<a href="https://colab.research.google.com/github/centeno/covid-impact-se/blob/master/step01_1_collection_grey_GoogleDrive_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [327]:

#!apt-get update # to update ubuntu to correctly run apt install


'''
# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3

'''


'\n# scraper\n!pip install selenium\n!apt install chromium-chromedriver\n!pip install webdriver-manager\n!cp /usr/lib/chromium-browser/chromedriver/usr/bin\n\n!apt-get update\n!apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\n!pip install selenium msticpy tld folium\n!pip pymongo pymongo[srv] dnspython\n\n# geocoder ipaddress\n!pip install msticpy\n\n# get domain\n!pip3 install tld\n\n\n# spacy\n!pip install -U pip setuptools wheel\n!pip install -U spacy\n#!python -m spacy download en\n#!python -m spacy download pt\n!python -m spacy download en_core_web_sm\n# spacy contextual Spell Check\n!pip install contextualSpellCheck\n!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3\n\n'

### Import das dependencias

In [328]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [329]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [330]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/01_grey/2021/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

2021-01-01.csv	2021-02-07.csv	2021-03-16.csv	2021-04-22.csv	2021-05-29.csv
2021-01-02.csv	2021-02-08.csv	2021-03-17.csv	2021-04-23.csv	2021-05-30.csv
2021-01-03.csv	2021-02-09.csv	2021-03-18.csv	2021-04-24.csv	2021-05-31.csv
2021-01-04.csv	2021-02-10.csv	2021-03-19.csv	2021-04-25.csv	2021-06-01.csv
2021-01-05.csv	2021-02-11.csv	2021-03-20.csv	2021-04-26.csv	2021-06-02.csv
2021-01-06.csv	2021-02-12.csv	2021-03-21.csv	2021-04-27.csv	2021-06-03.csv
2021-01-07.csv	2021-02-13.csv	2021-03-22.csv	2021-04-28.csv	2021-06-04.csv
2021-01-08.csv	2021-02-14.csv	2021-03-23.csv	2021-04-29.csv	2021-06-05.csv
2021-01-09.csv	2021-02-15.csv	2021-03-24.csv	2021-04-30.csv	2021-06-06.csv
2021-01-10.csv	2021-02-16.csv	2021-03-25.csv	2021-05-01.csv	2021-06-07.csv
2021-01-11.csv	2021-02-17.csv	2021-03-26.csv	2021-05-02.csv	2021-06-08.csv
2021-01-12.csv	2021-02-18.csv	2021-03-27.csv	2021-05-03.csv	2021-06-09.csv
2021-01-13.csv	2021-02-19.csv	2021-03-28.csv	2021-05-04.csv	2021-06-10.csv
2021-01-14.csv	2021-02-20

### Import chaves de api

In [331]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impactSE/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [332]:
GSE_RESULTS_CLASS = 'tF2Cxc' #'ZINbbc xpd O9g5cc uUPGi'
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

'''
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  d_before = (date + DateOffset(days = 2)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = " + "after:" + date.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', '').split('&sa')[0],
         
    }
    
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))
'''

def search(query, date):

  # set chrome_options for chrome driver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

  # define url de busca
  d = date.strftime('%Y%m%d')
  d_after = (date - DateOffset(days = 1)).date()
  d_before = (date + DateOffset(days = 1)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, d_after.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = "+"after:"+ d_after.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')

  # get url com driver
  driver.get(url)
  # passa o html para o BeautifulSoup
  soup = BeautifulSoup(driver.page_source,'html.parser') 
  # busca a tag principal das news do google earth engine
  items = soup.find_all('div',{'class':GSE_RESULTS_CLASS})

  # copia o conteudo da class 'tF2Cxc' para uma lista
  result = []
  print('counts : ' + str(len(items)))

  for each in items:
    try:
      header = each.find('div',{'class':'yuRUbf'}).find('h3').text
      summary = each.find('div',{'class':'IsZvec'}).find('span').text
      link = each.find('a', href=True)['href']

      
      result.append({'date' : date.strftime('%Y-%m-%d'),
                     'header' : header,
                     'summary': summary,
                     'link' : link})
    except:
      pass

  # cria um csv com a lista  
  df = pd.DataFrame(result)
  return df

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [333]:
# Scraper 1 (2020)
keyword = '"covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"'

### Merge csv's or SCRAPER new search

In [334]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])

  #get next data
  next_date = next_data(df)

  #get seach
  df_new = search(keyword, next_date)

  #get domain
  df_new = fetch_domain(df_new)

  #get hostname
  df_new = fetch_hostname(df_new)

  #get ipaddress
  df_new = fetch_ipaddress(df_new)

  #get geocoded ipaddress
  df_new = fetch_location(df_new)

  #get nlp contextualSpellCheck 
  df_new = fetch_nlp(df_new)

  #save df new
  save(df_new)

  #print new df
  df_new.head(2)
  
else:
  #get first date
  first_date = date(2021, 1, 1)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  df = fetch_ipaddress(df)
  #get geocoded ipaddress
  df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: use options instead of chrome_options


https://www.google.com/search?q="covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working" after:2021-07-02 and before:2021-07-03&sort=date:r:2021-07-01:2021-07-03&num=200
horizon = after:2021-07-01 and before:2021-07-03


counts : 36


,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
62,62,2021-07-01,RIIO-ED2 Business Plan 2023-2028 - Your Power ...,Workforce resilience. Expenditure to ensure WP...,https://yourpowerfuture.westernpower.co.uk/dow...,https,yourpowerfuture.westernpower.co.uk,/downloads-view/41172,NaN,NaN,westernpower.co.uk,5.57.63.77,GB,United Kingdom,England,Oxford,-1.275560,51.750000,204167.0,NaN,geolocation,{},5.57.63.77,RIIO-ED2 Business Plan 2023-2028 - Your Power ...,Workforce resilience. Expenditure to ensure WP...
63,63,2021-07-01,LONG TERM PLAN 2021/31 - Clutha District Council,"Jul 1, 2021 —",https://www.cluthadc.govt.nz/repository/librar...,https,www.cluthadc.govt.nz,/repository/libraries/id:2c0gik8bh17q9s5atec4/...,NaN,NaN,cluthadc.govt.nz,13.55.26.216,AU,Australia,New South Wales,Sydney,151.207108,-33.867142,16509.0,NaN,geolocation,{},13.55.26.216,LONG TERM PLAN 2021/31 - Clutha District Council,"Jul 1, 2021 —"
64,64,2021-07-01,T e x a s D e p a r t m e n t o f C r i m i n ...,"Jul 1, 2021 —",https://www.bidnet.com/bneattachments?/7042012...,https,www.bidnet.com,/bneattachments,/704201210.pdf,NaN,bidnet.com,205.205.237.204,CA,Canada,Quebec,Longueuil,-73.520462,45.519718,30180.0,NaN,geolocation,{},205.205.237.204,T e x a s D e p a r t m e n t o f C r i m i n ...,"Jul 1, 2021 —"
65,65,2021-07-01,EPC prospects - Oil Review Middle East,"Jul 1, 2021 —",https://www.oilreviewmiddleeast.com/aaccpp/Dig...,https,www.oilreviewmiddleeast.com,/aaccpp/DigitalMagDownload/ORME_4_2021.pdf,NaN,NaN,oilreviewmiddleeast.com,46.43.3.31,GB,United Kingdom,England,York,-1.070280,53.951389,35425.0,NaN,geolocation,{},46.43.3.31,EPC prospects - Oil Review Middle East,"Jul 1, 2021 —"
66,66,2021-07-01,Software that empowers local Takeaway Restaurants,"Jul 1, 2021 —",https://nyemission.dk/wp-content/uploads/Order...,https,nyemission.dk,/wp-content/uploads/OrderYOYO_-_Company_Descri...,NaN,NaN,nyemission.dk,217.195.176.194,DK,Denmark,Capital Region,Copenhagen,12.587830,55.680511,31027.0,NaN,geolocation,{},217.195.176.194,Software that empowers local Takeaway Restaurants,"Jul 1, 2021 —"


In [335]:
df_new.head()

,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-07-02,COVID's Impact on Agile Project Management | C...,"Jul 2, 2021 —",https://www.cioinsight.com/collaboration/agile...,https,www.cioinsight.com,/collaboration/agile-project-management/,,,cioinsight.com,141.193.213.21,US,United States,Texas,Austin,-97.742310,30.267200,209242.0,None,geolocation,{},141.193.213.21,COVID's Impact on Agile Project Management | C...,"Jul 2, 2021 —"
1,2021-07-02,COVID's Impact on Agile Project Management | C...,"Jul 2, 2021 —",https://www.cioinsight.com/collaboration/agile...,https,www.cioinsight.com,/collaboration/agile-project-management/,,,cioinsight.com,141.193.213.21,US,United States,Texas,Austin,-97.742310,30.267200,209242.0,None,geolocation,{},141.193.213.21,COVID's Impact on Agile Project Management | C...,"Jul 2, 2021 —"
2,2021-07-02,How Did Covid Impact Tech Job Market | JobTerix,"Jul 2, 2021 —",https://jobterix.com/posts/6/how-did-covid-imp...,https,jobterix.com,/posts/6/how-did-covid-impact-tech-job-market/,,,jobterix.com,65.8.249.33,US,United States,Florida,Flagami,-80.311958,25.762859,16509.0,None,geolocation,{},65.8.249.33,How Did Covid Impact Tech Job Market | JobTerix,"Jul 2, 2021 —"
3,2021-07-02,The Impact on Implementing Agile Methodology d...,The Covid-19 pandemic during 2020 and 2021 for...,https://www.dpublication.com/wp-content/upload...,https,www.dpublication.com,/wp-content/uploads/2021/07/2-9312.pdf,,,dpublication.com,72.52.251.254,US,United States,Michigan,Waverly,-84.632339,42.718868,32244.0,None,geolocation,{},72.52.251.254,The Impact on Implementing Agile Methodology d...,The Covid-19 pandemic during 2020 and 2021 for...
4,2021-07-02,IBM Power Systems (i/AIX) and AWS Hybrid Archi...,"By connecting IBM Power workloads to AWS, you ...",https://aws.amazon.com/marketplace/pp/prodview...,https,aws.amazon.com,/marketplace/pp/prodview-7vj3cq2balphq,,,amazon.com,13.35.107.75,US,United States,Florida,Flagami,-80.311958,25.762859,16509.0,None,geolocation,{},13.35.107.75,IBM Power Systems (i/AIX) and AWS Hybrid Archi...,"By connecting IBM Power workloads to AWS, you ..."
